# Compiling NIFTI files from tiff stack files

For the compatibility in the following ANTs registration,  
the images taken from the other side to the main angle are flipped and re-oriented.  

You can interactively set the annotation phrase.  
For example, if there are files like the following:  
- descSPIM_angle1_GFP_target_signal_some_notes.tif
- descSPIM_angle2_GFP_target_signal_some_notes.tif
- descSPIM_angle1_PI_nuclear_stain_some_notes.tif
- descSPIM_angle2_PI_nuclear_stain_some_notes.tif  

**To set the 'angle1' as the main angle, enter in the message box 'angle2'**.  
All the images that contains 'angle2' will be flipped both horizontally and along z axis, so that the compiled NIFTI files can be used in the following ANTs registration steps.

In [ ]:
"""
TIFF Stack to NIfTI Converter
------------------------------
This script provides utility functions to convert TIFF stacks to NIfTI format (.nii.gz).
It also includes functionalities to flip the TIFF stacks based on a given keyword, normalize and scale image values, and save both full-resolution and downsized NIfTI images.

Dependencies:
- os
- glob
- tifffile
- numpy
- nibabel
- skimage.transform
- multiprocessing
- concurrent.futures

Author: takakiom
Date: 09/19/2023
License: MIT License (refer to the LICENSE file in the root directory for more details)
"""

import os
import glob
import tifffile
import numpy as np
import nibabel as nb
from skimage.transform import rescale
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor


def load_tiff_stack(imgpath):
    """
    Load a TIFF stack from a given path.
    
    Parameters:
    - imgpath (str): Path to the TIFF image.
    
    Returns:
    - stack (numpy.ndarray): Loaded TIFF stack.
    """
    stack = tifffile.imread(imgpath)
    return stack


def flip_tiff_stack(stack):
    """
    Flip a TIFF stack along its last axis and the zeroth axis.
    
    Parameters:
    - stack (numpy.ndarray): TIFF stack to be flipped.
    
    Returns:
    - numpy.ndarray: Flipped TIFF stack.
    """
    return np.flip(np.flip(stack, axis=2), axis=0)


def normalize_and_scale(img):
    """
    Normalize an image to [0, 255] range and scale its values.
    
    Parameters:
    - img (numpy.ndarray): Image to be normalized and scaled.
    
    Returns:
    - numpy.ndarray: Normalized and scaled image.
    """
    img = (img - img.min()) / (img.max() - img.min())
    img = (img * 255).astype(np.uint8)
    return img


def save_nifti(stack, niftiname, spx, spy, spz):
    """
    Save an image stack as a NIfTI file with a specified spatial resolution.
    
    Parameters:
    - stack (numpy.ndarray): Image stack to be saved.
    - niftiname (str): Name of the output NIfTI file.
    - spx, spy, spz (float): Spatial resolutions in x, y, and z dimensions respectively.
    
    Returns:
    - None
    """
    stack = np.swapaxes(stack, 0, 2)
    nim = nb.Nifti1Image(stack, affine=None)
    aff = np.diag([-spx, -spy, spz, 1])
    nim.header.set_qform(aff, code=2)
    nim.to_filename(niftiname)


def process_folder(base_dir, filename, flip_key, spx, spy, spz):
    """
    Process TIFF stacks in a folder: load, optionally flip, and save as full-size and downsized NIfTI images.
    
    Parameters:
    - base_dir (str): Base directory containing TIFF stacks.
    - filename (str): Name of the TIFF file to be processed.
    - flip_key (str): Keyword indicating which TIFF stacks need to be flipped.
    - spx, spy, spz (float): Spatial resolutions in x, y, and z dimensions respectively.
    
    Returns:
    - None
    """
    file_name = os.path.splitext(os.path.basename(filename))[0]

    full_stack = load_tiff_stack(filename)
    if flip_key in file_name:
        file_name = file_name.replace(flip_key, f'Flipped{flip_key}')
        full_stack = flip_tiff_stack(full_stack)

    # Save full resolution nifti
    save_nifti(full_stack, os.path.join(base_dir, f'{file_name}_fullsize.nii.gz'), spx, spy, spz)
    
    # Save downsize nifti
    down_stack = full_stack[::ds_z, :, :]
    down_stack = rescale(down_stack, (1, ds_xy, ds_xy), anti_aliasing=True, channel_axis=None)
    down_stack = normalize_and_scale(down_stack)
    xy_scale = int(1/ds_xy)
    save_nifti(down_stack, os.path.join(base_dir, f'{file_name}_downsize.nii.gz'), spx*xy_scale, spy*xy_scale, spz*ds_z)


if __name__ == "__main__":
    base_dir = input("Enter the directory which contains the raw files.")
    flip_key = input("Enter the annotation for the angle180 (e.g., angle180, Angle180, angle2 etc.).")
    ds_xy = float(input('Enter the xy-plane downsizing scale. (e.g.,0.25 or 0.5. A 0.25 downsizing will resample original 4x4 pixels into one pixel.'))
    ds_z = int(input('Enter the downsizing scale for z axis. Deafault is four, which means extract one-in-four slices.'))
    # Define spatial resolutions
    spx = float(input('Enter X-axis resolution.'))
    spy = float(input('Enter Y-axis resolution.'))
    spz = float(input('Enter Z-axis resolution.'))


    file_list = glob.glob(os.path.join(base_dir, "*.tif"))

    with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
        executor.map(process_folder, [base_dir]*len(file_list), file_list, [flip_key]*len(file_list), [spx]*len(file_list), [spy]*len(file_list), [spz]*len(file_list))
